# Quais fatores econômicos ou sociais lavam a um maior sucesso eleitoral dos candidatos?

### Datasets Utilizados
* Candidatos
* Eleições
* Resultados de Eleições

## Importando Bibliotecas

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Dados

### Candidatos

In [ ]:
candidatos_df = pd.read_parquet('datasets/consulta_cand_2022/consulta_cand_2022_RS.parquet')

candidatos_df.head()

In [ ]:
candidatos_df.columns

In [23]:
CANDIDATOS_FEATURES = [
    'NM_CANDIDATO',
    'TP_ABRANGENCIA_ELEICAO',
    'DS_CARGO',
    'NR_TURNO',
    'NR_CANDIDATO',
    'SG_PARTIDO',
    'NR_PARTIDO',
    'CD_ELEICAO',
    'SG_UF',
    'SQ_CANDIDATO',
    'DT_NASCIMENTO',  # transformar em idade
    # variáveis categóricas interessantes
    'DS_GENERO',
    'DS_GRAU_INSTRUCAO',
    'DS_ESTADO_CIVIL',
    'DS_COR_RACA',
    'DS_OCUPACAO',
    'DS_SIT_TOT_TURNO',
]

In [24]:
candidatos = candidatos_df[CANDIDATOS_FEATURES]

### Despesas Contratadas Candidatos

Gastos de um candidato com sua campanha eleitoral.

In [25]:
despessas_contratadas_df = pd.read_parquet('datasets/prestacao_contas_2022/despesas_contratadas_candidatos_2022_RS.parquet')

In [ ]:
despessas_contratadas_df.head()

In [ ]:
despessas_contratadas_df.columns

In [28]:
DESPESAS_CONTRATA_FEATURES = [
    'SQ_CANDIDATO',
    'NR_PARTIDO',
    'SG_PARTIDO',
    'NM_PARTIDO',
    'CD_TIPO_FORNECEDOR',
    'DS_TIPO_FORNECEDOR',
    'CD_ESFERA_PART_FORNECEDOR',
    'DS_ESFERA_PART_FORNECEDOR',
    'SG_UF_FORNECEDOR',
    'CD_ORIGEM_DESPESA',
    'DS_ORIGEM_DESPESA',
    'DT_DESPESA',
    'DS_DESPESA',
    'SQ_DESPESA',
    'VR_DESPESA_CONTRATADA'
]



In [29]:
despesas_contratadas = despessas_contratadas_df[DESPESAS_CONTRATA_FEATURES]

### Despesas Pagas Candidatos

O que de fato foi pago pelo candidato de todas suas despesas eleitorais.

In [ ]:
despessas_pagas_df = pd.read_parquet('datasets/prestacao_contas_2022/despesas_pagas_candidatos_2022_RS.parquet')
despessas_pagas_df.head()

In [ ]:

despessas_pagas_df.columns

In [32]:
DESPESAS_PAGAS_FEATURES = [
    'SQ_PRESTADOR_CONTAS',
    'DT_PAGTO_DESPESA',
    'DS_DESPESA',
    'SQ_DESPESA',
    'VR_PAGTO_DESPESA'
]

In [33]:
despesas_pagas = despessas_pagas_df[DESPESAS_PAGAS_FEATURES]

### Resultados Eleições

Contagem dos votos de cada candidato no boletim de urna.

In [34]:
resultados_df = pd.read_parquet('datasets/boletins_urna_2022/bweb_1t_RS_051020221321.parquet')

In [ ]:
resultados_df.head()

In [ ]:
resultados_df.columns

In [37]:
RESULTADOS_FEATURES = [
    'NR_TURNO',
    'CD_ELEICAO',
    'NR_VOTAVEL',
    'SG_PARTIDO',
    'NR_PARTIDO',
    'CD_MUNICIPIO',
    'NR_ZONA',
    'NR_SECAO',
    'QT_VOTOS' # o que realmente queremos, computar agrupando por candidato, turno e eleição
]

In [38]:
resultados = resultados_df[RESULTADOS_FEATURES]

## Análise Exploratória de Dados (EDA)

### Candidatos

### Despesas

### Resultados

In [ ]:
votos_por_candidato = resultados_df.groupby(['NR_TURNO', 'CD_ELEICAO','DS_CARGO_PERGUNTA', 'NM_VOTAVEL', 'NR_VOTAVEL'])['QT_VOTOS'].sum().reset_index()

votos_por_candidato.head()

In [ ]:
top_10_votos = votos_por_candidato.sort_values(by='QT_VOTOS', ascending=False).head(10)

top_10_votos

#### Candidatos mais votados

In [41]:
cargos = ["Presidente", "Governador", "Senador", "Deputado Federal", "Deputado Estadual"]

top_10_votos_por_cargo = {}

for cargo in cargos:
    top_10_votos_por_cargo[cargo] = votos_por_candidato[votos_por_candidato['DS_CARGO_PERGUNTA'] == cargo].sort_values(by='QT_VOTOS', ascending=False).head(5)

In [ ]:
top_10_votos_por_cargo['Deputado Estadual']


In [ ]:


fig, axes = plt.subplots(2, 2, figsize=(20, 15))

axes = axes.flatten()

for i, cargo in enumerate(cargos):
    data = top_10_votos_por_cargo[cargo]
    labels = data['NM_VOTAVEL']
    sizes = data['QT_VOTOS']
    
    sns.barplot(x=labels, y=sizes, palette='viridis', hue=labels, ax=axes[i])
    axes[i].set_title(f'Candidatos mais votados - {cargo}')
    axes[i].set_xlabel('Candidatos')
    axes[i].set_ylabel('Numero de Votos')
    axes[i].tick_params(axis='x', rotation=90)

plt.tight_layout()
plt.show()

### Municipios com mais votos brancos e nulos

In [ ]:
abstencoes_df = resultados_df[resultados_df['NM_VOTAVEL'].isin(['Branco', 'Nulo'])]

abstencoes_por_municipio = abstencoes_df.groupby(['NM_MUNICIPIO', 'NM_VOTAVEL'])['QT_VOTOS'].sum().reset_index()

abstencoes_por_municipio = abstencoes_por_municipio.sort_values(by='QT_VOTOS', ascending=False)

top_5_cities = abstencoes_por_municipio.groupby('NM_MUNICIPIO')['QT_VOTOS'].sum().nlargest(5).index

top_5_abstencoes = abstencoes_por_municipio[abstencoes_por_municipio['NM_MUNICIPIO'].isin(top_5_cities)]

# Plot the data
plt.figure(figsize=(14, 7))
sns.barplot(x='NM_MUNICIPIO', y='QT_VOTOS', hue='NM_VOTAVEL', data=top_5_abstencoes, palette='viridis')
plt.title('Número de Abstenções por Município')
plt.xlabel('Município')
plt.ylabel('Número de Votos')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

### Partidos com mais votos (Deputado Federal e Estadual)

In [ ]:
partidos = [partido for partido in resultados_df['SG_PARTIDO'].unique() if partido != '#NULO#']


votos_deputados = resultados_df[resultados_df['DS_CARGO_PERGUNTA'].isin(['Deputado Federal', 'Deputado Estadual'])].query('SG_PARTIDO in @partidos')


votos_por_partido = votos_deputados.groupby(['NR_TURNO', 'CD_ELEICAO', 'SG_PARTIDO'])['QT_VOTOS'].sum().reset_index().sort_values(by='QT_VOTOS', ascending=False)

top_5_partidos = votos_por_partido.groupby('SG_PARTIDO')['QT_VOTOS'].sum().nlargest(5).index

top_5_votos_partidos = votos_por_partido[votos_por_partido['SG_PARTIDO'].isin(top_5_partidos)]


plt.figure(figsize=(14, 7))
sns.barplot(x='SG_PARTIDO', y='QT_VOTOS', hue='SG_PARTIDO', data=top_5_votos_partidos, palette='viridis')

plt.title('Número de Votos por Partido')
plt.xlabel('Partido')
plt.ylabel('Número de Votos')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()
